# Notebook to explore and rank CT anaesthetics jobs

In [1]:
import pandas as pd
import numpy as np
import googlemaps
import datetime as datetime
import json 
import re

In [2]:
df = pd.read_csv("PreferenceExtract-0110289789.csv")
df.shape
df.head()

,Current Assigned Rank,Preference,Start Date,Duration,Grade,Region,Sub-Region,Area(Sector),Dual Training Programme,Places Available,Description,Programme Name,Programme Number,Tags
0,X,East Midlands - North - ACCS 1,04/08/2021 00:00:00,36,Core Training Year 1,Health Education England East Midlands,Health Education England East Midlands - North,-,NaN,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN,NaN,NaN
1,X,East Midlands - North - ACCS 2,04/08/2021 00:00:00,36,Core Training Year 1,Health Education England East Midlands,Health Education England East Midlands - North,-,NaN,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN,NaN,NaN
2,X,East Midlands - North - ACCS 3,04/08/2021 00:00:00,36,Core Training Year 1,Health Education England East Midlands,Health Education England East Midlands - North,-,NaN,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN,NaN,NaN
3,X,East Midlands - North - Core 1,04/08/2021 00:00:00,24,Core Training Year 1,Health Education England East Midlands,Health Education England East Midlands - North,-,NaN,3,East Midlands North rotation commencing at Que...,NaN,NaN,NaN
4,X,East Midlands - North - Core 2,04/08/2021 00:00:00,24,Core Training Year 1,Health Education England East Midlands,Health Education England East Midlands - North,-,NaN,3,East Midlands North rotation commencing at Not...,NaN,NaN,NaN


255 line of jobs, however some are for more than one post
Now let's see how many places are actually availible in the areas I could concievably commute to

In [3]:
df['Places Available'].sum()

474

In [4]:
df.describe()

,Duration,Dual Training Programme,Places Available,Programme Name,Programme Number,Tags
count,255.000000,0.0,255.000000,0.0,0.0,0.0
mean,29.270588,NaN,1.858824,NaN,NaN,NaN
std,5.967210,NaN,3.235457,NaN,NaN,NaN
min,24.000000,NaN,0.000000,NaN,NaN,NaN
25%,24.000000,NaN,1.000000,NaN,NaN,NaN
50%,24.000000,NaN,1.000000,NaN,NaN,NaN
75%,36.000000,NaN,2.000000,NaN,NaN,NaN
max,36.000000,NaN,35.000000,NaN,NaN,NaN


Some empty columns so lets get trid of them

In [5]:
df.drop(columns=["Dual Training Programme","Programme Name", "Programme Number", "Tags", "Current Assigned Rank", "Grade", "Start Date"], inplace=True)

In [6]:
long  = df.loc[df['Region'].isin(['Health Education England East Midlands','Health Education England East of England','Health Education England London' ])]
print(long.shape)
long['Places Available'].sum()

(128, 7)


165

Ok so only 165 jobs that could concievably be commuteable

In [7]:
long.head()

,Preference,Duration,Region,Sub-Region,Area(Sector),Places Available,Description
0,East Midlands - North - ACCS 1,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...
1,East Midlands - North - ACCS 2,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...
2,East Midlands - North - ACCS 3,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...
3,East Midlands - North - Core 1,24,Health Education England East Midlands,Health Education England East Midlands - North,-,3,East Midlands North rotation commencing at Que...
4,East Midlands - North - Core 2,24,Health Education England East Midlands,Health Education England East Midlands - North,-,3,East Midlands North rotation commencing at Not...


In [8]:
long.iloc[1,6]

'East Midlands North ACCS Anaesthetics. Rotation N21. Aug 21 - Acute Med Kings Mill; Feb 22 - Emer Med Derby; Aug 22 - Anaes Derby; Feb 23 - ICM Derby.  All posts indicative, subject to change'

Lets clean up the description with some for loops, not optimal but it is a small dataset

In [9]:
long['Duration'].replace()
extr = long['Description']
extr = extr.str.replace("Due to the implementation of the new Anaesthetics Curriculum in August 2021, the remainder of this programme will be confirmed with Training Programme Directors upon appointment to the programme",
                ".")
extr = extr.str.replace("Due to the implementation of the new Anaesthetics Curriculum in August 2021, the remainder of this programme will be confirmed at a later date",
                ".")
extr = extr.str.replace("Due to the implementation of the new Anaesthetics Curriculum in August 2021, the remainder of this programme will be confirmed with Training Programme Directors upon appointment to programme",
                ".")
extr = extr.str.replace("Due to the implementation of the new Anaesthetics Curriculum in August 2021, the full programme will be confirmed with Training Programme Directors upon appointment to the programme",
                ".")
extr = extr.str.replace("/.","").replace(u'\xa0', u' ')
extr = extr.str.replace(",","")

long.loc[:,['Description']] = extr
long.loc[:,['newcolumn']] = np.nan

long

<ipython-input-9-670964c5ada2>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  extr = extr.str.replace("/.","").replace(u'\xa0', u' ')
/home/george/Documents/Development/.anaconda3/envs/CTapp/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/george/Documents/Development/.anaconda3/envs/CTapp/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,Preference,Duration,Region,Sub-Region,Area(Sector),Places Available,Description,newcolumn
0,East Midlands - North - ACCS 1,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN
1,East Midlands - North - ACCS 2,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN
2,East Midlands - North - ACCS 3,36,Health Education England East Midlands,Health Education England East Midlands - North,-,1,East Midlands North ACCS Anaesthetics. Rotatio...,NaN
3,East Midlands - North - Core 1,24,Health Education England East Midlands,Health Education England East Midlands - North,-,3,East Midlands North rotation commencing at Que...,NaN
4,East Midlands - North - Core 2,24,Health Education England East Midlands,Health Education England East Midlands - North,-,3,East Midlands North rotation commencing at Not...,NaN
...,...,...,...,...,...,...,...,...
150,London - South London - Core - 5,24,Health Education England London,Health Education England South London,-,1,London South West - CT1 Croydon University Hos...,NaN
151,London - South London - Core - 6,24,Health Education England London,Health Education England South London,-,4,London South East - CT1 Guy's & St Thomas' NHS...,NaN
152,London - South London - Core - 7,24,Health Education England London,Health Education England South London,-,3,London South East - CT1 Queen Elizabeth Hospit...,NaN
153,London - South London - Core - 8,24,Health Education England London,Health Education England South London,-,1,London South East - CT1 University Hospital Le...,NaN


Now lets work out some commute lengths

In [10]:
def regex(year , s):
    s=s.replace("/.","").replace(u'\xa0', u' ')
    res = re.search(r"(?:CT"+year+"[ ,:;])[^0-9]+(?= Hospital)",s)
    if res != None:
        res= res.group()[3:].replace("at ", "")+" Hospital"
        return res
    if year == "1":
        res = re.search(r"(commencing at)[^0-9]+(?= Hospital)",s)
        if res != None:
            res= res.group()[13:].replace("at ", "")+" Hospital"
            print(res)
            return res
    print(s)
    return np.nan

long['CT1 Hostpital'] = long['Description'].apply(lambda x: regex("1",x))
long.to_csv("longlist_text.csv")

East Midlands North ACCS Anaesthetics. Rotation M21. Aug 21 - Emer Med Lincoln County; Feb 22 - Acute Med Kings Mill; Aug 22 - Anaes Nottingham City; Feb 23 - ICM Nottingham City. All posts indicative subject to change
East Midlands North ACCS Anaesthetics. Rotation N21. Aug 21 - Acute Med Kings Mill; Feb 22 - Emer Med Derby; Aug 22 - Anaes Derby; Feb 23 - ICM Derby.  All posts indicative subject to change
East Midlands North ACCS Anaesthetics. Rotation P21. Aug 21 - Emer Med Kings Mill; Feb 22 - Acute Med Kings Mill; Aug 22 - Anaes QMC; Feb 23 - ICM QMC. All posts indicative subject to change
East Midlands North rotation commencing at Queens Medical Centre (6 months) - indicative subject to change
 Nottingham City Hospital
 Kings Mill Hospital
 Lincoln County Hospital
 Chesterfield Royal Hospital
 Queens Hospital
East Midlands South ACCS Anaesthetics.  Rotation Anaes 1. Aug 21 - Emer Med Kettering General; Feb 22 - Acute Med Leicester Royal; Aug 22 - Anaes Leicester Royal; Feb 23 - IT

<ipython-input-10-331c1405b476>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long['CT1 Hostpital'] = long['Description'].apply(lambda x: regex("1",x))


In [21]:
with open('./creds/apikey.txt') as f0:
    google_api_key = str(f0.readline().rstrip())
with open('./files/locations.json') as f2:
    json_location = json.load(f2)
    
def cal_commutes(arr,location):
    gmaps = googlemaps.Client(key=google_api_key)
    commute_dict = {}
    
    for i in arr.unique():
        commute_dict[i] = [0,0]
        print("calculating commute for: " + str(i))
        geocode_result = gmaps.geocode(i)
        try:
            coord = geocode_result[0]['geometry']['location']
            #row['lat'] = coord['lat']
            #row['lng'] = coord['lng']
        except:
            print('could not get coords')
            #row['lat'] = np.nan
            #row['lng'] = np.nan
    
        # Look up an address with reverse geocoding
        #reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))
        # Request directions via public transit
        tommorrow = datetime.datetime.today() + datetime.timedelta(days=1)
        tomorrow_morn = tommorrow.replace(hour=8, minute=0)
    
        try:
            directions_result = gmaps.directions(i,
                                     location,
                                     mode="transit",
                                     departure_time=tomorrow_morn)
        
        
            duration = directions_result[0]['legs'][0]['duration']['value']
            commute_dict[i][0] = str(duration//60)
        except:
            print("An exception occurred on " + str(i) + ' to ' + str(location))
            commute_dict[i][0] = np.nan
        # turn result into minutes
        print(commute_dict[i][0])
        try:
            directions_result = gmaps.directions(i,
                                     location,
                                     mode="driving",
                                     departure_time=tomorrow_morn)
    
    
            duration = directions_result[0]['legs'][0]['duration']['value']
            commute_dict[i][1] = str(duration//60)
        except:
            print("An exception occurred on " + str(i) + ' to ' + str(location))
            commute_dict[i][1] = np.nan
                                  
        
    return commute_dict
    
    
def apply_commute(x , dic, i):
    return dic[x][i]





In [22]:
print(json_location["home"])
commutes = cal_commutes(long['CT1 Hostpital'], json_location["home"])
print(commutes)

51.590272590693075,-0.023473245276590206
calculating commute for: nan
An exception occurred on nan to 51.590272590693075,-0.023473245276590206
nan
An exception occurred on nan to 51.590272590693075,-0.023473245276590206
calculating commute for:  Nottingham City Hospital
193
calculating commute for:  Kings Mill Hospital
223
calculating commute for:  Lincoln County Hospital
An exception occurred on  Lincoln County Hospital to 51.590272590693075,-0.023473245276590206
nan
An exception occurred on  Lincoln County Hospital to 51.590272590693075,-0.023473245276590206
calculating commute for:  Chesterfield Royal Hospital
188
calculating commute for:  Queens Hospital
An exception occurred on  Queens Hospital to 51.590272590693075,-0.023473245276590206
nan
An exception occurred on  Queens Hospital to 51.590272590693075,-0.023473245276590206
calculating commute for:  Leicester General Hospital
151
calculating commute for:  Royal Derby Hospital
162
calculating commute for:  Kettering General Hospi

In [23]:
commutes_df = pd.DataFrame.from_dict(commutes, orient='index')

In [24]:
commutes_df.to_csv("commutes.csv")

In [25]:
long['CT1 Transit'] = long['CT1 Hostpital'].apply(lambda x: apply_commute(x,commutes,0))
long['CT1 Driving'] = long['CT1 Hostpital'].apply(lambda x: apply_commute(x,commutes,1))

<ipython-input-25-dfab7190185f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long['CT1 Transit'] = long['CT1 Hostpital'].apply(lambda x: apply_commute(x,commutes,0))
<ipython-input-25-dfab7190185f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long['CT1 Driving'] = long['CT1 Hostpital'].apply(lambda x: apply_commute(x,commutes,1))


In [26]:
long.to_csv("longlist_text.csv")

In [27]:
long.to_csv("longlist_text.csv", sep="@")